# 공공데이터 분석 연습(전국 신규 민간 아파트 분양가격 동향 분석)

* 자료1. 전국 평균 분양가격(13년 9월 ~ 15년 8월)
* 자료2. 주택도시보증공사 전국 평균 분양가격(19년 12월)

## 데이터 가져오기

In [5]:
# 라이브러리 가져오기
import pandas as pd

# 경로설정
path = '/Users/parkjinwoo/Desktop/public_data_analysis/open-data-analysis-basic-master/'
# 자료1 가져오기
df_latest = pd.read_csv(path + '주택도시보증공사_전국 평균 분양가격(2019년 12월).csv', encoding = 'cp949')
# 자료2 가져오기
df_former = pd.read_csv(path + '전국 평균 분양가격(2013년 9월부터 2015년 8월까지).csv', encoding = 'cp949')

## 데이터 구성 확인하기

In [6]:
# df_latest # 전체 데이터 확인
# df_latest.head() #데이터의 상위 5개 확인
# df_latest.shape #행,열 갯수 확인

In [7]:
df_latest.info() # 분양가격에 결측지가 있음을 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4335 entries, 0 to 4334
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   지역명      4335 non-null   object
 1   규모구분     4335 non-null   object
 2   연도       4335 non-null   int64 
 3   월        4335 non-null   int64 
 4   분양가격(㎡)  4058 non-null   object
dtypes: int64(2), object(3)
memory usage: 169.5+ KB


In [8]:
df_former.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   지역        17 non-null     object
 1   2013년12월  17 non-null     int64 
 2   2014년1월   17 non-null     int64 
 3   2014년2월   17 non-null     int64 
 4   2014년3월   17 non-null     int64 
 5   2014년4월   17 non-null     int64 
 6   2014년5월   17 non-null     int64 
 7   2014년6월   17 non-null     int64 
 8   2014년7월   17 non-null     int64 
 9   2014년8월   17 non-null     int64 
 10  2014년9월   17 non-null     int64 
 11  2014년10월  17 non-null     int64 
 12  2014년11월  17 non-null     int64 
 13  2014년12월  17 non-null     int64 
 14  2015년1월   17 non-null     int64 
 15  2015년2월   17 non-null     int64 
 16  2015년3월   17 non-null     int64 
 17  2015년4월   17 non-null     int64 
 18  2015년5월   17 non-null     int64 
 19  2015년6월   17 non-null     int64 
 20  2015년7월   17 non-null     int64 
 21  2015년8월   17 non-n

## 결측치 제거

In [9]:
# 결측치 확인
df_latest.isnull().sum()

지역명          0
규모구분         0
연도           0
월            0
분양가격(㎡)    277
dtype: int64

In [10]:
# 결측치를 다른 값으로 바꾸기1(fillna 이용)
df_latest['분양가격(㎡)'].fillna(0)

0       5841
1       5652
2       5882
3       5721
4       5879
        ... 
4330    3882
4331       0
4332    3898
4333       0
4334    3601
Name: 분양가격(㎡), Length: 4335, dtype: object

In [11]:
df_latest['분양가격(㎡)'].astype('int64') # 공백으로 처리된 부분은 0으로 바뀌지 않았음. 다른 방법 사용

ValueError: invalid literal for int() with base 10: '  '

In [12]:
# 결측지 바꾸기2 (pd.to_numeric이용. 수로 바꾸는 함수)
df_latest['분양가격(㎡)'] = pd.to_numeric(df_latest['분양가격(㎡)'], errors = 'coerce') # coerce : 오류값들은 NaN으로 대체

In [13]:
df_latest.info() # NaN은 자료형이 float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4335 entries, 0 to 4334
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역명      4335 non-null   object 
 1   규모구분     4335 non-null   object 
 2   연도       4335 non-null   int64  
 3   월        4335 non-null   int64  
 4   분양가격(㎡)  3957 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 169.5+ KB


## 데이터 전처리
* 규모구분 -> 전용면적 
* 분양가격(㎡) -> 분양가격(평)

In [14]:
# 단위를 통일시키기(㎡당 가격 -> 평당 가격)
df_latest['분양가격(평)'] = df_latest['분양가격(㎡)'] * 3.3

In [15]:
# 규모구분 -> 전용면적으로 변환
# 필요없는 단어들은 삭제함으로써 용량을 줄일 수 있다

df_latest['전용면적'] = df_latest['규모구분'].str.replace('전용면적','')
df_latest['전용면적'] = df_latest['전용면적'].str.replace('초과','~')
df_latest['전용면적'] = df_latest['전용면적'].str.replace('이하','')
df_latest['전용면적'] = df_latest['전용면적'].str.strip()

In [16]:
df_latest = df_latest.drop(columns = ['규모구분'])

In [17]:
df_latest.head()

,지역명,연도,월,분양가격(㎡),분양가격(평),전용면적
0,서울,2015,10,5841.0,19275.3,전체
1,서울,2015,10,5652.0,18651.6,60㎡
2,서울,2015,10,5882.0,19410.6,60㎡~ 85㎡
3,서울,2015,10,5721.0,18879.3,85㎡~ 102㎡
4,서울,2015,10,5879.0,19400.7,102㎡~


## 평균분양가격 구하기
* groupby : 결과값이 리스트형태로 반환
* pivot_table : 결과값이 데이터프레임형태로 반환

### 지역별 평균분양가격 구하기

In [18]:
df_latest.groupby('지역명')['분양가격(평)'].mean()

지역명
강원     7890.750000
경기    13356.895200
경남     9268.778138
경북     8376.536515
광주     9951.535821
대구    11980.895455
대전    10253.333333
부산    12087.121200
서울    23599.976400
세종     9796.516456
울산    10014.902013
인천    11915.320732
전남     7565.316532
전북     7724.235484
제주    11241.276712
충남     8233.651883
충북     7634.655600
Name: 분양가격(평), dtype: float64

In [19]:
df_latest.pivot_table(index = '지역명', values = '분양가격(평)')

,분양가격(평)
지역명,
강원,7890.750000
경기,13356.895200
경남,9268.778138
경북,8376.536515
광주,9951.535821
대구,11980.895455
대전,10253.333333
부산,12087.121200
서울,23599.976400


### 전용면적별 평균분양가격 구하기

In [20]:
df_latest.groupby('전용면적')['분양가격(평)'].mean()

전용면적
102㎡~        11517.705634
60㎡          10375.137421
60㎡~ 85㎡     10271.040071
85㎡~ 102㎡    11097.599573
전체           10276.086207
Name: 분양가격(평), dtype: float64

### 지역별 전용면적의 평균분양가격 구하기

In [21]:
df_latest.groupby(['지역명','전용면적'])['분양가격(평)']

In [22]:
df_latest.pivot_table(index = '지역명', columns = '전용면적', values = '분양가격(평)')

전용면적,102㎡~,60㎡,60㎡~ 85㎡,85㎡~ 102㎡,전체
지역명,,,,,
강원,8311.380000,7567.098000,7485.588000,8749.557143,7477.536000
경기,14771.790000,13251.744000,12523.566000,13677.774000,12559.602000
경남,10358.363265,8689.175000,8618.676000,10017.612000,8658.672000
경북,9157.302000,7883.172000,8061.372000,8773.814634,8078.532000
광주,11041.532432,9430.666667,9910.692000,9296.100000,9903.630000
대구,13087.338000,11992.068000,11778.690000,11140.642857,11771.298000
대전,14876.871429,9176.475000,9711.372000,9037.430769,9786.018000
부산,13208.250000,11353.782000,11864.820000,12072.588000,11936.166000
서울,23446.038000,23212.794000,22786.830000,25943.874000,22610.346000


In [23]:
df_latest.pivot_table(index = '지역명', columns = '전용면적', values = '분양가격(평)').transpose() # transpose()함수를 통해 행과 열의 위치를 바꿀 수 있음

지역명,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
전용면적,,,,,,,,,,,,,,,,,
102㎡~,8311.380000,14771.790,10358.363265,9157.302000,11041.532432,13087.338000,14876.871429,13208.250,23446.038,10106.976000,9974.448000,14362.030435,8168.4900,8193.57000,10522.787234,8689.169388,8195.352
60㎡,7567.098000,13251.744,8689.175000,7883.172000,9430.666667,11992.068000,9176.475000,11353.782,23212.794,9323.927027,9202.106897,11241.318000,7210.1700,7609.93200,14022.221053,7911.156000,7103.118
60㎡~ 85㎡,7485.588000,12523.566,8618.676000,8061.372000,9910.692000,11778.690000,9711.372000,11864.820,22786.830,9775.458000,10502.531707,11384.406000,7269.2400,7271.35200,10621.314000,7818.954000,7264.488
85㎡~ 102㎡,8749.557143,13677.774,10017.612000,8773.814634,9296.100000,11140.642857,9037.430769,12072.588,25943.874,9847.926000,8861.007692,11527.560000,7908.8625,8275.78125,10709.082353,9120.045000,8391.306
전체,7477.536000,12559.602,8658.672000,8078.532000,9903.630000,11771.298000,9786.018000,11936.166,22610.346,9805.422000,10492.712195,11257.026000,7283.5620,7292.60400,10784.994000,7815.324000,7219.014


### 연도별, 지역별 평균분양가격 구하기

In [24]:
df_latest.groupby(['연도','지역명'])['분양가격(평)'].mean()

연도    지역명
2015  강원      7188.060000
      경기     11060.940000
      경남      8459.220000
      경북      7464.160000
      광주      7916.700000
                 ...     
2019  전남      8219.275862
      전북      8532.260000
      제주     11828.469231
      충남      8748.840000
      충북      7970.875000
Name: 분양가격(평), Length: 85, dtype: float64

## 15년 이전 데이터 전처리

In [25]:
df_former.head()

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,...,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,...,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443
1,부산,8111,8111,9078,8965,9402,9501,9453,9457,9411,...,9208,9208,9204,9235,9279,9327,9345,9515,9559,9581
2,대구,8080,8080,8077,8101,8267,8274,8360,8360,8370,...,8439,8253,8327,8416,8441,8446,8568,8542,8542,8795
3,인천,10204,10204,10408,10408,10000,9844,10058,9974,9973,...,10020,10020,10017,9876,9876,9938,10551,10443,10443,10449
4,광주,6098,7326,7611,7346,7346,7523,7659,7612,7622,...,7752,7748,7752,7756,7861,7914,7877,7881,8089,8231


In [38]:
# 15년 이후 데이터와 통일시키기
# 컬럼을 인덱스로 변환 : melt

df_former_melt = df_former.melt(id_vars = '지역', var_name = '날짜', value_name = '분양가격(평)')

In [39]:
df_former_melt.head()

,지역,날짜,분양가격(평)
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [40]:
# 컬럼의 이름 변경하기
df_former_melt.columns = ['지역명','날짜','분양가격(평)']

In [41]:
df_former_melt.head()

,지역명,날짜,분양가격(평)
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [ ]:
# apply함수를 이용한 연도와 월 구분하기

In [31]:
def parse_year(date):
    year = date.split('년')[0]
    year = int(year)
    return year

def parse_month(date):
    month = date.split('년')[1].replace('월','')
    month = int(month)
    return month

In [42]:
df_former_melt['연도'] = df_former_melt['날짜'].apply(parse_year)
df_former_melt['월'] = df_former_melt['날짜'].apply(parse_month)

In [50]:
df_former_melt.head()

,지역명,날짜,분양가격(평),연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


In [51]:
df_former_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   지역명      357 non-null    object
 1   날짜       357 non-null    object
 2   분양가격(평)  357 non-null    int64 
 3   연도       357 non-null    int64 
 4   월        357 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 14.1+ KB


In [53]:
cols1 = ['지역명','연도','월','분양가격(평)']
df_former_final = df_former_melt[cols1]

In [55]:
df_former_final.head()

,지역명,연도,월,분양가격(평)
0,서울,2013,12,18189
1,부산,2013,12,8111
2,대구,2013,12,8080
3,인천,2013,12,10204
4,광주,2013,12,6098
